In [1]:
import os
import itertools
import re
import numpy as np
import pandas as pd
# PyTorch
import torch

In [2]:
import sys
sys.path.append('../src/')

%load_ext autoreload
%autoreload 2
# Importing our custom module(s)
import losses
import utils

In [3]:
dataset_path = '/cluster/tufts/hugheslab/eharve06/CIFAR-10'
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_CIFAR-10'
lr_0s = [0.1, 0.01, 0.001, 0.0001]
ns = [10, 100, 1000, 10000, 50000]
ns = [1000]
prior_path = '/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior'
prior_type = 'nonlearned'
random_states = [1001, 2001, 3001]
wandb_project = 'tuned_CIFAR-10'
weight_decays = [0.01, 0.001, 0.0001, 1e-05, 1e-06, 0.0]
count = -1
for lr_0, n, random_state, weight_decay in itertools.product(lr_0s, ns, random_states, weight_decays):
    count += 1
    model_name = f'{prior_type}_lr_0={lr_0}_n={n}_random_state={random_state}_weight_decay={weight_decay}'
    print(f'    "python ../src/main_CIFAR-10.py --bb_weight_decay={weight_decay} --clf_weight_decay={weight_decay} --dataset_path=\'{dataset_path}\' --experiments_path=\'{experiments_path}\' --lr_0={lr_0} --n={n} --model_name=\'{model_name}\' --prior_path=\'{prior_path}\' --random_state={random_state} --tune --wandb --wandb_project=\'{wandb_project}\'"')
print(count)

    "python ../src/main_CIFAR-10.py --bb_weight_decay=0.01 --clf_weight_decay=0.01 --dataset_path='/cluster/tufts/hugheslab/eharve06/CIFAR-10' --experiments_path='/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_CIFAR-10' --lr_0=0.1 --n=1000 --model_name='nonlearned_lr_0=0.1_n=1000_random_state=1001_weight_decay=0.01' --prior_path='/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior' --random_state=1001 --tune --wandb --wandb_project='tuned_CIFAR-10'"
    "python ../src/main_CIFAR-10.py --bb_weight_decay=0.001 --clf_weight_decay=0.001 --dataset_path='/cluster/tufts/hugheslab/eharve06/CIFAR-10' --experiments_path='/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_CIFAR-10' --lr_0=0.1 --n=1000 --model_name='nonlearned_lr_0=0.1_n=1000_random_state=1001_weight_decay=0.001' --prior_path='/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior' --random_state=1001 --tune --wandb --wandb_project='tuned_CIFAR-10'"
    "python ../src/main_CIFAR-10.

In [4]:
dataset_path = '/cluster/tufts/hugheslab/eharve06/CIFAR-10'
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_CIFAR-10'
lr_0s = [0.1, 0.01, 0.001, 0.0001]
ns = [1000]
prior_path = '/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior'
prior_type = 'adapted'
random_states = [1001, 2001, 3001]
wandb_project = 'tuned_CIFAR-10'
weight_decays = [0.01, 0.001, 0.0001, 1e-05, 1e-06, 0.0]
count = -1
for lr_0, n, random_state, weight_decay in itertools.product(lr_0s, ns, random_states, weight_decays):
    count += 1
    model_name = f'{prior_type}_lr_0={lr_0}_n={n}_random_state={random_state}_weight_decay={weight_decay}'
    print(f'    "python ../src/main_CIFAR-10.py --bb_weight_decay={weight_decay} --clf_weight_decay={weight_decay} --dataset_path=\'{dataset_path}\' --experiments_path=\'{experiments_path}\' --lr_0={lr_0} --n={n} --model_name=\'{model_name}\' --prior_path=\'{prior_path}\' --prior_type=\'{prior_type}\' --random_state={random_state} --tune --wandb --wandb_project=\'{wandb_project}\'"')
print(count)

    "python ../src/main_CIFAR-10.py --bb_weight_decay=0.01 --clf_weight_decay=0.01 --dataset_path='/cluster/tufts/hugheslab/eharve06/CIFAR-10' --experiments_path='/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_CIFAR-10' --lr_0=0.1 --n=1000 --model_name='adapted_lr_0=0.1_n=1000_random_state=1001_weight_decay=0.01' --prior_path='/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior' --prior_type='adapted' --random_state=1001 --tune --wandb --wandb_project='tuned_CIFAR-10'"
    "python ../src/main_CIFAR-10.py --bb_weight_decay=0.001 --clf_weight_decay=0.001 --dataset_path='/cluster/tufts/hugheslab/eharve06/CIFAR-10' --experiments_path='/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_CIFAR-10' --lr_0=0.1 --n=1000 --model_name='adapted_lr_0=0.1_n=1000_random_state=1001_weight_decay=0.001' --prior_path='/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior' --prior_type='adapted' --random_state=1001 --tune --wandb --wandb_project='tuned_CIFA

In [5]:
bb_weight_decays = np.logspace(0, 9, num=10)
clf_weight_decays = [0.01, 0.001, 0.0001, 1e-05, 1e-06, 0.0]
dataset_path = '/cluster/tufts/hugheslab/eharve06/CIFAR-10'
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_CIFAR-10'
lr_0s = [0.1, 0.01, 0.001, 0.0001]
ns = [1000]
prior_path = '/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior'
prior_type = 'LearnedPriorLR'
random_states = [1001, 2001, 3001]
wandb_project = 'tuned_CIFAR-10'
count = -1
for bb_weight_decay, clf_weight_decay, lr_0, n, random_state in itertools.product(bb_weight_decays, clf_weight_decays, lr_0s, ns, random_states):
    count += 1
    model_name = f'{prior_type}_bb_weight_decay={bb_weight_decay}_clf_weight_decay={clf_weight_decay}_lr_0={lr_0}_n={n}_random_state={random_state}'
    print(f'    "python ../src/main.py --bb_weight_decay={bb_weight_decay} --clf_weight_decay={clf_weight_decay} --dataset_path=\'{dataset_path}\' --experiments_path=\'{experiments_path}\' --lr_0={lr_0} --n={n} --model_name=\'{model_name}\' --prior_path=\'{prior_path}\' --prior_type=\'{prior_type}\' --random_state={random_state} --tune --wandb --wandb_project=\'{wandb_project}\'"')
print(count)

    "python ../src/main.py --bb_weight_decay=1.0 --clf_weight_decay=0.01 --dataset_path='/cluster/tufts/hugheslab/eharve06/CIFAR-10' --experiments_path='/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_CIFAR-10' --lr_0=0.1 --n=1000 --model_name='LearnedPriorLR_bb_weight_decay=1.0_clf_weight_decay=0.01_lr_0=0.1_n=1000_random_state=1001' --prior_path='/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior' --prior_type='LearnedPriorLR' --random_state=1001 --tune --wandb --wandb_project='tuned_CIFAR-10'"
    "python ../src/main.py --bb_weight_decay=1.0 --clf_weight_decay=0.01 --dataset_path='/cluster/tufts/hugheslab/eharve06/CIFAR-10' --experiments_path='/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_CIFAR-10' --lr_0=0.1 --n=1000 --model_name='LearnedPriorLR_bb_weight_decay=1.0_clf_weight_decay=0.01_lr_0=0.1_n=1000_random_state=2001' --prior_path='/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior' --prior_type='LearnedPriorLR' --random_

In [6]:
def get_df(path):
    df = pd.read_csv(path, index_col='Unnamed: 0')
    return df

def get_val_nll(df):
    return df.val_or_test_nll.values[-1]

def get_val_acc1(df):
    if 'val_or_test_acc' in df.columns:
        return df.val_or_test_acc.values[-1] if df.val_or_test_acc.values[-1] < 1.0 else df.val_or_test_acc.values[-1]/100
    elif 'val_or_test_acc1' in df.columns:
        return df.val_or_test_acc1.values[-1] if df.val_or_test_acc1.values[-1] < 1.0 else df.val_or_test_acc1.values[-1]/100

In [7]:
def get_best_hyperparams(
    bb_weight_decays, 
    clf_weight_decays, 
    experiments_path, # Gets best hyperparameters from experiments_path
    lr_0s, 
    ns, 
    prior_type, # Gets best hyperparameters for a single prior_type
    random_states, 
    weight_decays
):
    hyperparam_names = ['bb_weight_decay', 'clf_weight_decay', 'lr_0', 'n', 'random_state', 'weight_decay']
    columns = ['model_name', 'n', 'prior_type', 'random_state', 'val_acc1', 'val_nll']
    df = pd.DataFrame(columns=columns)
    
    for hyperparams in itertools.product(bb_weight_decays, clf_weight_decays, lr_0s, ns, random_states, weight_decays):
        model_name = f'{prior_type}_' + '_'.join(f'{hyperparam_name}={hyperparam}' for hyperparam_name, hyperparam in zip(hyperparam_names, hyperparams) if hyperparam is not None)
        bb_weight_decay, clf_weight_decay, lr_0, n, random_state, weight_decay = hyperparams
        if not os.path.exists(f'{experiments_path}/{model_name}.csv'):
            print(model_name)
            pass
        else:
            temp_df = get_df(f'{experiments_path}/{model_name}.csv')
            val_acc1 = get_val_acc1(temp_df)
            val_nll = get_val_nll(temp_df)
            df.loc[df.shape[0]] = [model_name, n, prior_type, random_state, val_acc1, val_nll]
                    
    min_indices = df.groupby(['n', 'random_state'])['val_nll'].idxmin()
    df = df.loc[min_indices].reset_index(drop=True)
    #max_indices = df.groupby(['n', 'random_state'])['val_acc1'].idxmax()
    #df = df.loc[max_indices].reset_index(drop=True)
                    
    return df

def get_results(
    df,
    experiments_path
):
    columns = ['model_name', 'n', 'prior_type', 'random_state', 'test_acc1', 'test_nll']
    results_df = pd.DataFrame(columns=columns)
    
    for row_index, row in df.iterrows():
        model_name = row.model_name
        prior_type, n, random_state = row.prior_type, row.n, row.random_state
        if not os.path.exists(f'{experiments_path}/{model_name}.csv'):
            #print(model_name)
            pass
        else:
            temp_df = get_df(f'{experiments_path}/{model_name}.csv')
            test_acc1 = get_val_acc1(temp_df)
            test_nll = get_val_nll(temp_df)
            condition = (results_df['n']==n)&(results_df['random_state']==random_state)
            if len(results_df[condition]) == 0:
                results_df.loc[results_df.shape[0]] = [model_name, n, prior_type, random_state, test_acc1, test_nll]
    
    return results_df

In [8]:
dataset_path = '/cluster/tufts/hugheslab/eharve06/CIFAR-10'
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_CIFAR-10'
lr_0s = [0.1, 0.01, 0.001, 0.0001]
ns = [10, 100, 1000, 10000, 50000]
prior_path = '/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior'
prior_type = 'nonlearned'
random_states = [1001, 2001, 3001]
wandb_project = 'tuned_CIFAR-10'
weight_decays = [0.01, 0.001, 0.0001, 1e-05, 1e-06, 0.0]

std_prior = get_best_hyperparams([None], [None], experiments_path, lr_0s, ns, prior_type, random_states, weight_decays)
std_prior

,model_name,n,prior_type,random_state,val_acc1,val_nll
0,nonlearned_lr_0=0.01_n=10_random_state=1001_we...,10,nonlearned,1001,0.0000,2.826736
1,nonlearned_lr_0=0.0001_n=10_random_state=2001_...,10,nonlearned,2001,0.0000,3.600672
2,nonlearned_lr_0=0.001_n=10_random_state=3001_w...,10,nonlearned,3001,0.0000,2.696253
3,nonlearned_lr_0=0.01_n=100_random_state=1001_w...,100,nonlearned,1001,0.7000,0.951838
4,nonlearned_lr_0=0.001_n=100_random_state=2001_...,100,nonlearned,2001,0.5000,1.355964
5,nonlearned_lr_0=0.001_n=100_random_state=3001_...,100,nonlearned,3001,0.6500,0.983005
6,nonlearned_lr_0=0.0001_n=1000_random_state=100...,1000,nonlearned,1001,0.7550,0.670592
7,nonlearned_lr_0=0.01_n=1000_random_state=2001_...,1000,nonlearned,2001,0.8200,0.803543
8,nonlearned_lr_0=0.01_n=1000_random_state=3001_...,1000,nonlearned,3001,0.8650,0.434508
9,nonlearned_lr_0=0.01_n=10000_random_state=1001...,10000,nonlearned,1001,0.9265,0.262783


In [9]:
dataset_path = '/cluster/tufts/hugheslab/eharve06/CIFAR-10'
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/retrained_CIFAR-10'
prior_path = '/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior'
wandb_project = 'retrained_CIFAR-10'

pattern = re.compile(r'(\w+)_lr_0=([\d.]+)_n=(\d+)_random_state=(\d+)_weight_decay=([\d.]+(?:e[-+]?\d+)?)')
for row_index, row in std_prior.iterrows():
    model_name = row.model_name
    match = pattern.match(model_name)
    prior_type, lr_0, n, random_state, weight_decay = match.groups()
    if not os.path.exists(f'{experiments_path}/{model_name}.csv'):
        print(f'    "python ../src/main_CIFAR-10.py --bb_weight_decay={weight_decay} --clf_weight_decay={weight_decay} --dataset_path=\'{dataset_path}\' --experiments_path=\'{experiments_path}\' --lr_0={lr_0} --n={n} --model_name=\'{model_name}\' --prior_path=\'{prior_path}\' --prior_type=\'{prior_type}\' --random_state={random_state} --save --wandb --wandb_project=\'{wandb_project}\'"')

In [10]:
dataset_path = '/cluster/tufts/hugheslab/eharve06/CIFAR-10'
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_CIFAR-10'
lr_0s = [0.1, 0.01, 0.001, 0.0001]
ns = [10, 100, 1000, 10000, 50000]
prior_path = '/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior'
prior_type = 'adapted'
random_states = [1001, 2001, 3001]
wandb_project = 'tuned_CIFAR-10'
weight_decays = [0.01, 0.001, 0.0001, 1e-05, 1e-06, 0.0]

learned_prior_iso = get_best_hyperparams([None], [None], experiments_path, lr_0s, ns, prior_type, random_states, weight_decays)
learned_prior_iso

,model_name,n,prior_type,random_state,val_acc1,val_nll
0,adapted_lr_0=0.1_n=10_random_state=1001_weight...,10,adapted,1001,0.0000,2.377469
1,adapted_lr_0=0.1_n=10_random_state=2001_weight...,10,adapted,2001,0.0000,2.474465
2,adapted_lr_0=0.1_n=10_random_state=3001_weight...,10,adapted,3001,0.0000,2.341610
3,adapted_lr_0=0.001_n=100_random_state=1001_wei...,100,adapted,1001,0.7000,0.983326
4,adapted_lr_0=0.001_n=100_random_state=2001_wei...,100,adapted,2001,0.5000,1.374754
5,adapted_lr_0=0.001_n=100_random_state=3001_wei...,100,adapted,3001,0.6500,0.972365
6,adapted_lr_0=0.1_n=1000_random_state=1001_weig...,1000,adapted,1001,0.8350,0.523390
7,adapted_lr_0=0.1_n=1000_random_state=2001_weig...,1000,adapted,2001,0.8150,0.667335
8,adapted_lr_0=0.01_n=1000_random_state=3001_wei...,1000,adapted,3001,0.9100,0.392827
9,adapted_lr_0=0.1_n=10000_random_state=1001_wei...,10000,adapted,1001,0.9335,0.199625


In [11]:
dataset_path = '/cluster/tufts/hugheslab/eharve06/CIFAR-10'
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/retrained_CIFAR-10'
prior_path = '/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior'
wandb_project = 'retrained_CIFAR-10'

pattern = re.compile(r'(\w+)_lr_0=([\d.]+)_n=(\d+)_random_state=(\d+)_weight_decay=([\d.]+(?:e[-+]?\d+)?)')
for row_index, row in learned_prior_iso.iterrows():
    model_name = row.model_name
    match = pattern.match(model_name)
    prior_type, lr_0, n, random_state, weight_decay = match.groups()
    if not os.path.exists(f'{experiments_path}/{model_name}.csv'):
        print(f'    "python ../src/main_CIFAR-10.py --bb_weight_decay={weight_decay} --clf_weight_decay={weight_decay} --dataset_path=\'{dataset_path}\' --experiments_path=\'{experiments_path}\' --lr_0={lr_0} --n={n} --model_name=\'{model_name}\' --prior_path=\'{prior_path}\' --prior_type=\'{prior_type}\' --random_state={random_state} --save --wandb --wandb_project=\'{wandb_project}\'"')

In [12]:
bb_weight_decays = np.logspace(0, 9, num=10)
clf_weight_decays = [0.01, 0.001, 0.0001, 1e-05, 1e-06, 0.0]
dataset_path = '/cluster/tufts/hugheslab/eharve06/CIFAR-10'
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_CIFAR-10'
lr_0s = [0.1, 0.01, 0.001, 0.0001]
ns = [10, 100, 1000, 10000, 50000]
prior_path = '/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior'
prior_type = 'LearnedPriorLR'
random_states = [1001, 2001, 3001]
wandb_project = 'tuned_CIFAR-10'

learned_prior_lr = get_best_hyperparams(bb_weight_decays, clf_weight_decays, experiments_path, lr_0s, ns, prior_type, random_states, [None])
learned_prior_lr

,model_name,n,prior_type,random_state,val_acc1,val_nll
0,LearnedPriorLR_bb_weight_decay=1000000.0_clf_w...,10,LearnedPriorLR,1001,0.0000,2.511933
1,LearnedPriorLR_bb_weight_decay=1000000.0_clf_w...,10,LearnedPriorLR,2001,0.0000,2.973418
2,LearnedPriorLR_bb_weight_decay=10000000.0_clf_...,10,LearnedPriorLR,3001,0.0000,2.329212
3,LearnedPriorLR_bb_weight_decay=100000.0_clf_we...,100,LearnedPriorLR,1001,0.6000,1.021263
4,LearnedPriorLR_bb_weight_decay=1.0_clf_weight_...,100,LearnedPriorLR,2001,0.7500,0.906630
5,LearnedPriorLR_bb_weight_decay=1000000.0_clf_w...,100,LearnedPriorLR,3001,0.7000,0.967850
6,LearnedPriorLR_bb_weight_decay=100.0_clf_weigh...,1000,LearnedPriorLR,1001,0.8750,0.436516
7,LearnedPriorLR_bb_weight_decay=100.0_clf_weigh...,1000,LearnedPriorLR,2001,0.8300,0.609634
8,LearnedPriorLR_bb_weight_decay=10.0_clf_weight...,1000,LearnedPriorLR,3001,0.8950,0.299455
9,LearnedPriorLR_bb_weight_decay=10.0_clf_weight...,10000,LearnedPriorLR,1001,0.9325,0.228510


In [13]:
dataset_path = '/cluster/tufts/hugheslab/eharve06/CIFAR-10'
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/retrained_CIFAR-10'
prior_path = '/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior'
wandb_project = 'retrained_CIFAR-10'

pattern = re.compile(r'(\w+)_bb_weight_decay=([\d.]+)_clf_weight_decay=([\d.]+(?:e[-+]?\d+)?)_lr_0=([\d.]+)_n=(\d+)_random_state=(\d+)')
for row_index, row in learned_prior_lr.iterrows():
    model_name = row.model_name
    match = pattern.match(model_name)
    prior_type, bb_weight_decay, clf_weight_decay, lr_0, n, random_state = match.groups()
    if not os.path.exists(f'{experiments_path}/{model_name}.csv'):
        print(f'    "python ../src/main_CIFAR-10.py --bb_weight_decay={bb_weight_decay} --clf_weight_decay={clf_weight_decay} --dataset_path=\'{dataset_path}\' --experiments_path=\'{experiments_path}\' --lr_0={lr_0} --n={n} --model_name=\'{model_name}\' --prior_path=\'{prior_path}\' --prior_type=\'{prior_type}\' --random_state={random_state} --save --wandb --wandb_project=\'{wandb_project}\'"')

In [14]:
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/retrained_CIFAR-10'
std_prior_results = get_results(std_prior, experiments_path)
learned_prior_iso_results = get_results(learned_prior_iso, experiments_path)
learned_prior_lr_results = get_results(learned_prior_lr, experiments_path)
results = pd.concat([std_prior_results, learned_prior_iso_results, learned_prior_lr_results])
results

,model_name,n,prior_type,random_state,test_acc1,test_nll
0,nonlearned_lr_0=0.01_n=10_random_state=1001_we...,10,nonlearned,1001,0.1317,3.200835
1,nonlearned_lr_0=0.0001_n=10_random_state=2001_...,10,nonlearned,2001,0.2621,2.097741
2,nonlearned_lr_0=0.001_n=10_random_state=3001_w...,10,nonlearned,3001,0.2669,2.124110
3,nonlearned_lr_0=0.01_n=100_random_state=1001_w...,100,nonlearned,1001,0.5301,1.753938
4,nonlearned_lr_0=0.001_n=100_random_state=2001_...,100,nonlearned,2001,0.5987,1.273124
5,nonlearned_lr_0=0.001_n=100_random_state=3001_...,100,nonlearned,3001,0.5575,1.404902
6,nonlearned_lr_0=0.0001_n=1000_random_state=100...,1000,nonlearned,1001,0.7818,0.689700
7,nonlearned_lr_0=0.01_n=1000_random_state=2001_...,1000,nonlearned,2001,0.8181,0.790316
8,nonlearned_lr_0=0.01_n=1000_random_state=3001_...,1000,nonlearned,3001,0.8275,0.655546
9,nonlearned_lr_0=0.01_n=10000_random_state=1001...,10000,nonlearned,1001,0.9372,0.226423


In [15]:
grouped_results = results.groupby(['n', 'prior_type']).agg(lambda x: tuple(x))
columns = ['test_acc1', 'test_nll']
for column in columns:
    grouped_results[f'{column}_mean'] = grouped_results[column].apply(lambda item: np.mean(item))
    grouped_results[f'{column}_std'] = grouped_results[column].apply(lambda item: np.std(item))
    grouped_results[f'{column}_min'] = grouped_results[column].apply(lambda item: np.min(item))
    grouped_results[f'{column}_max'] = grouped_results[column].apply(lambda item: np.max(item))
grouped_results = grouped_results.reset_index()
grouped_results

,n,prior_type,model_name,random_state,test_acc1,test_nll,test_acc1_mean,test_acc1_std,test_acc1_min,test_acc1_max,test_nll_mean,test_nll_std,test_nll_min,test_nll_max
0,10,LearnedPriorLR,(LearnedPriorLR_bb_weight_decay=1000000.0_clf_...,"(1001, 2001, 3001)","(0.1708, 0.1883, 0.11250000000000003)","(2.303623052215577, 2.3892940402984637, 3.3865...",0.157200,0.032405,0.1125,0.1883,2.693154,0.491547,2.303623,3.386544
1,10,adapted,(adapted_lr_0=0.1_n=10_random_state=1001_weigh...,"(1001, 2001, 3001)","(0.2339000254869461, 0.175599992275238, 0.2116...","(2.244244303131104, 2.346000033950806, 2.32525...",0.207067,0.024025,0.1756,0.2339,2.305165,0.043903,2.244244,2.346000
2,10,nonlearned,(nonlearned_lr_0=0.01_n=10_random_state=1001_w...,"(1001, 2001, 3001)","(0.1316999942064285, 0.2621000111103058, 0.266...","(3.200834867477418, 2.0977412963867184, 2.1241...",0.220233,0.062633,0.1317,0.2669,2.474229,0.513901,2.097741,3.200835
3,100,LearnedPriorLR,(LearnedPriorLR_bb_weight_decay=100000.0_clf_w...,"(1001, 2001, 3001)","(0.5640000000000001, 0.5900000000000001, 0.564...","(1.439711721801758, 1.2922731227874755, 1.4419...",0.572667,0.012257,0.5640,0.5900,1.391326,0.070047,1.292273,1.441993
4,100,adapted,(adapted_lr_0=0.001_n=100_random_state=1001_we...,"(1001, 2001, 3001)","(0.5476000308990479, 0.5982000231742859, 0.555...","(1.4966200017929077, 1.3010373809814455, 1.406...",0.567267,0.022140,0.5476,0.5982,1.401535,0.079938,1.301037,1.496620
5,100,nonlearned,(nonlearned_lr_0=0.01_n=100_random_state=1001_...,"(1001, 2001, 3001)","(0.5300999879837036, 0.5986999869346619, 0.557...","(1.7539382343292234, 1.2731240596771238, 1.404...",0.562100,0.028194,0.5301,0.5987,1.477322,0.202861,1.273124,1.753938
6,1000,LearnedPriorLR,(LearnedPriorLR_bb_weight_decay=100.0_clf_weig...,"(1001, 2001, 3001)","(0.8386000000000002, 0.8326000000000002, 0.834...","(0.5820776025295256, 0.5649791242122649, 0.578...",0.835300,0.002486,0.8326,0.8386,0.575300,0.007416,0.564979,0.582078
7,1000,adapted,(adapted_lr_0=0.1_n=1000_random_state=1001_wei...,"(1001, 2001, 3001)","(0.8163000345230103, 0.820099949836731, 0.8209...","(0.6228910679817198, 0.6203525841712954, 0.782...",0.819133,0.002037,0.8163,0.8210,0.675372,0.076022,0.620353,0.782873
8,1000,nonlearned,(nonlearned_lr_0=0.0001_n=1000_random_state=10...,"(1001, 2001, 3001)","(0.7818000316619873, 0.8181000351905823, 0.827...","(0.6896996079444885, 0.7903160516738894, 0.655...",0.809133,0.019705,0.7818,0.8275,0.711854,0.057206,0.655546,0.790316
9,10000,LearnedPriorLR,(LearnedPriorLR_bb_weight_decay=10.0_clf_weigh...,"(1001, 2001, 3001)","(0.9407000000000002, 0.9337999999999999, 0.939...","(0.2186949087619782, 0.231239442539215, 0.2095...",0.937867,0.002949,0.9338,0.9407,0.219824,0.008895,0.209538,0.231239


In [16]:
grouped_results[['n', 'prior_type', 'test_acc1_mean', 'test_acc1_min', 'test_acc1_max']]

,n,prior_type,test_acc1_mean,test_acc1_min,test_acc1_max
0,10,LearnedPriorLR,0.157200,0.1125,0.1883
1,10,adapted,0.207067,0.1756,0.2339
2,10,nonlearned,0.220233,0.1317,0.2669
3,100,LearnedPriorLR,0.572667,0.5640,0.5900
4,100,adapted,0.567267,0.5476,0.5982
5,100,nonlearned,0.562100,0.5301,0.5987
6,1000,LearnedPriorLR,0.835300,0.8326,0.8386
7,1000,adapted,0.819133,0.8163,0.8210
8,1000,nonlearned,0.809133,0.7818,0.8275
9,10000,LearnedPriorLR,0.937867,0.9338,0.9407


In [17]:
grouped_results[['n', 'prior_type', 'test_nll_mean', 'test_nll_min', 'test_nll_max']]

,n,prior_type,test_nll_mean,test_nll_min,test_nll_max
0,10,LearnedPriorLR,2.693154,2.303623,3.386544
1,10,adapted,2.305165,2.244244,2.346000
2,10,nonlearned,2.474229,2.097741,3.200835
3,100,LearnedPriorLR,1.391326,1.292273,1.441993
4,100,adapted,1.401535,1.301037,1.496620
5,100,nonlearned,1.477322,1.273124,1.753938
6,1000,LearnedPriorLR,0.575300,0.564979,0.582078
7,1000,adapted,0.675372,0.620353,0.782873
8,1000,nonlearned,0.711854,0.655546,0.790316
9,10000,LearnedPriorLR,0.219824,0.209538,0.231239


In [18]:
# Get model names for interpolations from paper
print(results[(results.n==1000)&(results.prior_type=='nonlearned')].model_name.to_list())
print(results[(results.n==1000)&(results.prior_type=='adapted')].model_name.to_list())
print(results[(results.n==1000)&(results.prior_type=='LearnedPriorLR')].model_name.to_list())

['nonlearned_lr_0=0.0001_n=1000_random_state=1001_weight_decay=0.0001', 'nonlearned_lr_0=0.01_n=1000_random_state=2001_weight_decay=1e-05', 'nonlearned_lr_0=0.01_n=1000_random_state=3001_weight_decay=0.001']
['adapted_lr_0=0.1_n=1000_random_state=1001_weight_decay=0.01', 'adapted_lr_0=0.1_n=1000_random_state=2001_weight_decay=0.01', 'adapted_lr_0=0.01_n=1000_random_state=3001_weight_decay=1e-05']
['LearnedPriorLR_bb_weight_decay=100.0_clf_weight_decay=0.0_lr_0=0.01_n=1000_random_state=1001', 'LearnedPriorLR_bb_weight_decay=100.0_clf_weight_decay=0.001_lr_0=0.1_n=1000_random_state=2001', 'LearnedPriorLR_bb_weight_decay=10.0_clf_weight_decay=0.001_lr_0=0.1_n=1000_random_state=3001']


In [19]:
# Get model names for interpolations from paper
print(results[(results.n==50000)&(results.prior_type=='LearnedPriorLR')].model_name.to_list())

['LearnedPriorLR_bb_weight_decay=10.0_clf_weight_decay=0.01_lr_0=0.01_n=50000_random_state=1001', 'LearnedPriorLR_bb_weight_decay=10.0_clf_weight_decay=0.01_lr_0=0.01_n=50000_random_state=2001', 'LearnedPriorLR_bb_weight_decay=10.0_clf_weight_decay=0.01_lr_0=0.01_n=50000_random_state=3001']
